 метод который слою кварталов и землепользования назначает тег из группы zoning

In [1]:
import geopandas as gpd
from collections import defaultdict
from tqdm import tqdm

In [3]:
PZZ_gdf = gpd.read_file("./data/PZZ/PZZ.geojson")

In [4]:
PZZ_gdf = PZZ_gdf.to_crs(epsg=3857)

In [5]:
def filter_columns(PZZ):
    # Оставляем только нужные столбцы: геометрия, вид, наименование вида
    filtered_gdf = PZZ[['geometry', 'Зона']]    
    return filtered_gdf

In [6]:
PZZ_type_gdf = filter_columns(PZZ_gdf)

In [ ]:
PZZ_type_gdf 

In [7]:
def add_zonning_tag(PZZ_zoning_gdf):
    # Словарь для сопоставления зоны с тегом
    zone_mapping = {
        "RESIDENTIAL": ['Т1Ж1', 'Т1Ж2-1', 'Т1Ж2-2', 'Т2Ж1', 'Т2ЖД2', 'Т3Ж1', 'Т3Ж2', 'Т3ЖД3'],
        "PUBLIC_AND_BUSINESS" : ['ТД1-1_1', 'ТД1-1_2', 'ТД1-2_1', 'ТД1-2_2', 'ТД1-3', 'ТД2_1', 'ТД2_2', 'ТД3'],
        "INDUSTRIAL" : ['ТИП1', 'ТИ2', 'ТИП3', 'ТИП4', 'ТПД1_1', 'ТПД1_2', 'ТПД1_3', 'ТПД2_1', 'ТПД2_2', 'ТПД2_3', 'ТОЭЗ'],
        "ENGINEERING_AND_TRANSPORTATION" : ['ТИ1-1', 'ТИ1-2', 'ТИ2', 'ТИ3', 'ТИ4-1', 'ТИ4-2', 'ТУ'],
        "AGRICULTURAL" : ['ТС1', 'ТС2'],
        "RECREATIONAL" : ['ТР0-1', 'ТР0-2', 'ТР1', 'ТР2', 'ТР3-1', 'ТР3-2', 'ТР3-3', 'ТР4', 'ТР5-1', 'ТР5-2'],
        "SPETIAL_PURPOSE" : ['ТК1', 'ТК2', 'ТК3']
    }

    # Функция для назначения тега на основе значения в столбце "Зона"
    def get_zonning_tag(zone_value):
        for tag, zone_list in zone_mapping.items():
            if zone_value in zone_list:
                return tag
        return 'Unknown'  # Если совпадение не найдено

    # Проверим наличие столбца 'Зона' в данных
    if 'Зона' not in PZZ_zoning_gdf.columns:
        raise ValueError("Столбец 'Зона' не найден в GeoDataFrame")

    # Применяем функцию ко всем строкам в столбце 'Зона' и создаем новый столбец 'ZONING_tag'
    # Используем .loc[] для безопасного изменения
    PZZ_zoning_gdf.loc[:, 'ZONING_tag'] = PZZ_zoning_gdf['Зона'].apply(get_zonning_tag)

    return PZZ_zoning_gdf

In [8]:
PZZ_zoning_gdf = add_zonning_tag(PZZ_type_gdf)

c:\repos\master-plan-requirements\.venv\lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
#PZZ_zoning_gdf.to_file("PZZ_zoning_gdf.geojson", driver="GeoJSON")

In [122]:
PZZ_zoning_gdf

,geometry,Зона,ZONING_tag
0,"POLYGON ((3368210.843 8378494.833, 3368224.245...",,Unknown
1,"POLYGON ((3368297.001 8378654.669, 3368307.295...",,Unknown
2,"POLYGON ((3376792.279 8380567.984, 3376804.641...",,Unknown
3,"POLYGON ((3376889.367 8380638.420, 3377002.782...",,Unknown
4,"POLYGON ((3376607.073 8380435.761, 3376626.740...",,Unknown
...,...,...,...
8422,"POLYGON ((3358227.095 8396511.354, 3358276.054...",,Unknown
8423,"MULTIPOLYGON (((3358542.015 8396548.875, 33585...",,Unknown
8424,"POLYGON ((3357707.823 8396612.299, 3357726.983...",,Unknown
8425,"POLYGON ((3359348.393 8401121.685, 3359358.978...",ТР1,RECREATIONAL


КВАРТАЛЫ

In [15]:
#загружаем геометрию кварталов
blocks_gdf = gpd.read_file("./data/blocks_relation_tagger.geojson")

In [16]:
blocks_gdf  = blocks_gdf.to_crs(epsg=3857)

In [ ]:
blocks_gdf

In [17]:
print(PZZ_zoning_gdf.columns)

Index(['geometry', 'Зона', 'ZONING_tag', 'ZONE'], dtype='object')


In [18]:
def intersect(blocks_gdf, PZZ_zoning_gdf):
    # Создаем новый столбец для добавления информации о зоне
    blocks_gdf['ZONE'] = None

    # Создаем словарь для хранения зон для каждого квартала
    zoning_dict = defaultdict(list)

    # Используем tqdm для отображения прогресса
    for idx, block in tqdm(blocks_gdf.iterrows(), total=len(blocks_gdf), desc="Processing blocks"):
        # Получаем геометрию квартала
        block_geometry = block['geometry']
        
        # Находим пересечения с геометриями зон
        intersecting_zones = PZZ_zoning_gdf[PZZ_zoning_gdf.geometry.intersects(block_geometry)]
        
        # Если есть пересечения, то собираем все соответствующие теги
        if not intersecting_zones.empty:
            for _, zone in intersecting_zones.iterrows():
                zoning_tag = zone['ZONING_tag']
                zoning_dict[idx].append(zoning_tag)
    
    # Теперь заполняем столбец ZONE в блоках
    for idx, zoning_tags in zoning_dict.items():
        if len(zoning_tags) == 1:
            blocks_gdf.at[idx, 'ZONE'] = zoning_tags[0]
        else:
            blocks_gdf.at[idx, 'ZONE'] = dict.fromkeys(zoning_tags)  # Записываем как словарь

    return blocks_gdf

In [14]:
PZZ_blocks = intersect(PZZ_zoning_gdf, blocks_gdf)

c:\repos\master-plan-requirements\.venv\lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
Processing blocks:   0%|          | 0/7162 [00:00<?, ?it/s]


KeyError: 'ZONING_tag'

In [ ]:
PZZ_blocks

рассчитать процент пересеяения зон для ТЭП 

In [2]:
from my_package.models import Tag, Group, GROUPS_TAGS

In [3]:
Group.FUNCTIONAL_ZONE

<Group.FUNCTIONAL_ZONE: 'Функциональная зона'>